In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 87.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.3 MB/s eta 0:00:00


#SQUAD dataset
QA dataset: https://www.kaggle.com/datasets/stanfordu/stanford-question-answering-dataset

#Dataset Preprocessing
학습에 용이하도록 데이터셋의 포맷을 변환  
paragraph를 하나의 list에 묶고 paragraph index, question과 answer index에 대한 pair list로 구성

In [3]:
import json

train_file_path = "/content/drive/MyDrive/nlp-open-tutorial/6일차_배포/dataset/train-v1.1.json"
test_file_path = "/content/drive/MyDrive/nlp-open-tutorial/6일차_배포/dataset/dev-v1.1.json"

train_data = json.load(open(train_file_path,"r"))
test_data = json.load(open(test_file_path,"r"))

#paragraph를 list에 저장하고 모든 data pair를 list에 저장함으로써 사용하기 쉽게만듦, 각 pair는 paragraph index, question, answer로 구성
train_paras = []
train_pairs = []
for p_index, i in enumerate(train_data["data"][0]["paragraphs"]):
  train_paras.append(i["context"])
  for j in i["qas"]:
    pair = (p_index, j["question"], j["answers"][0]["text"])
    train_pairs.append(pair)

test_paras = []
test_pairs = []
for p_index, i in enumerate(test_data["data"][0]["paragraphs"]):
  test_paras.append(i["context"])
  for j in i["qas"]:
    pair = (p_index, j["question"], j["answers"][0]["text"])
    test_pairs.append(pair)

print(train_paras[0])
print(train_pairs[0])
print()
print(test_paras[0])
print(test_pairs[0])
print()
print("train_paras: ",len(train_paras))
print("train_pairs: ",len(train_pairs))
print()
print("test_paras: ",len(test_paras))
print("test_pairs: ",len(test_pairs))

Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.
(0, 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?', 'Saint Bernadette Soubirous')

Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos de

#Dataset

In [8]:
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer
import torch

#start token, end token의 index를 찾아주는 함수
def find_sub_list(ans_ids,input_ids):
    results=[]
    ans_ids_len=len(ans_ids)
    for ind in (i for i,e in enumerate(input_ids) if e==ans_ids[0]):
        if input_ids[ind:ind+ans_ids_len]==ans_ids:
            results.append((ind,ind+ans_ids_len-1))
    return results

class myDataset(Dataset):
  def __init__(self, paras, pairs) -> None:
      super().__init__()
      self.paras = paras
      self.pairs = pairs
      self.tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

  def __len__(self):
      return len(self.pairs)

  def __getitem__(self, index):
      data_pair = self.pairs[index]
      paragraph = self.paras[data_pair[0]]
      question = data_pair[1]
      answer = data_pair[2]

      #question과 paragraph를 concatenate 하여 tokenization
      data = self.tokenizer(paragraph, question, max_length=512, padding='max_length', truncation=True)

      #answer를 tokenization
      tokenized_answer = self.tokenizer.convert_tokens_to_ids(self.tokenizer.tokenize(answer))
      # tokenized_answer = self.tokenizer(answer)["input_ids"][1:-1] 이것과 같은 결과(의미)

      #data에서 answer의 position 찾기
      ans = find_sub_list(tokenized_answer, data.input_ids)

      #position을 못찾으면 정답을 0이라 가정
      if len(ans) == 0:
        start_token_index = 0
        end_token_index = 0
      else:
        start_token_index = ans[0][0]
        end_token_index = ans[0][1]

      #BERT의 input들을 각각 tensor로 변환해서 반환
      input_ids = torch.IntTensor(data.input_ids)
      token_type_ids = torch.IntTensor(data.token_type_ids)
      attention_mask = torch.IntTensor(data.attention_mask)

      return input_ids, token_type_ids, attention_mask, start_token_index, end_token_index

train_dataset = myDataset(train_paras, train_pairs)
test_dataset = myDataset(test_paras, test_pairs)

batch_size = 3
train_dataloader = DataLoader(train_dataset, batch_size = batch_size)
test_dataloader = DataLoader(test_dataset, batch_size = batch_size)

for i in train_dataset:
  input_ids, token_type_ids, attention_mask, start_token_index, end_token_index=i
  print("input_ids: ",input_ids)
  print("token_type_ids: ",token_type_ids)
  print("attention_mask: ",attention_mask)
  print("start_token_index:", start_token_index)
  print("end_token_index:", end_token_index)
  break
print()

for batch in train_dataloader:
  input_ids, token_type_ids, attention_mask, target_start, target_end= batch
  print("batch_input_ids: ",input_ids.shape)
  print("batch_token_type_ids: ",token_type_ids.shape)
  print("batch_attention_mask: ",attention_mask.shape)
  print("batch_start_token_index: ",target_start.shape)
  print("batch_end_token_index: ",target_end.shape)
  break

input_ids:  tensor([  101,  6549,  2135,  1010,  1996,  2082,  2038,  1037,  3234,  2839,
         1012, 10234,  1996,  2364,  2311,  1005,  1055,  2751,  8514,  2003,
         1037,  3585,  6231,  1997,  1996,  6261,  2984,  1012,  3202,  1999,
         2392,  1997,  1996,  2364,  2311,  1998,  5307,  2009,  1010,  2003,
         1037,  6967,  6231,  1997,  4828,  2007,  2608,  2039, 14995,  6924,
         2007,  1996,  5722,  1000,  2310,  3490,  2618,  4748,  2033, 18168,
         5267,  1000,  1012,  2279,  2000,  1996,  2364,  2311,  2003,  1996,
        13546,  1997,  1996,  6730,  2540,  1012,  3202,  2369,  1996, 13546,
         2003,  1996, 24665, 23052,  1010,  1037, 14042,  2173,  1997,  7083,
         1998,  9185,  1012,  2009,  2003,  1037, 15059,  1997,  1996, 24665,
        23052,  2012, 10223, 26371,  1010,  2605,  2073,  1996,  6261,  2984,
        22353,  2135,  2596,  2000,  3002, 16595,  9648,  4674,  2061, 12083,
         9711,  2271,  1999,  8517,  1012,  2012,  1

##Model

In [9]:
from torch import nn
from transformers import BertModel

class MyModel(nn.Module):
  def __init__(self) -> None:
      super().__init__()
      self.bert = BertModel.from_pretrained('bert-base-uncased')
      self.linear_start = nn.Linear(768, 1)
      self.linear_end = nn.Linear(768, 1)


  def forward(self, input_ids, token_type_ids, attention_mask):
      output = self.bert(input_ids, token_type_ids, attention_mask)
      start_logit = self.linear_start(output.last_hidden_state)
      end_logit = self.linear_end(output.last_hidden_state)

      return start_logit.squeeze(), end_logit.squeeze()

# 잘 작동하는지 확인

model = MyModel().cuda()

for batch in train_dataloader:
  input_ids, token_type_ids, attention_mask, target_start, target_end= batch
  input_ids = input_ids.cuda()
  token_type_ids = token_type_ids.cuda()
  attention_mask =attention_mask.cuda()
  output = model(input_ids, token_type_ids, attention_mask)
  print(output[0].shape,output[1].shape)
  break

torch.Size([3, 512]) torch.Size([3, 512])


#Train and test

In [10]:
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
from tqdm.auto import tqdm

model = MyModel().cuda()

optimizer = Adam(model.parameters(), lr=0.0001)
lf = nn.CrossEntropyLoss()

for e in range(100):
  print("\nepoch ", e)
  epoch_loss = 0
  train_start_correct = 0
  train_end_correct = 0

  model.train()

  for batch in tqdm(train_dataloader):
    optimizer.zero_grad()

    #data GPU에 올려주기
    input_ids, token_type_ids, attention_mask, target_start, target_end= batch
    input_ids = input_ids.cuda()
    token_type_ids = token_type_ids.cuda()
    attention_mask = attention_mask.cuda()
    target_start = target_start.cuda()
    target_end = target_end.cuda()

    #결과 도출 및 정답수 연산
    start_logit, end_logit = model(input_ids, token_type_ids, attention_mask)

    start_pred_label = torch.argmax(start_logit, dim=-1)
    end_pred_label = torch.argmax(end_logit, dim=-1)

    train_start_correct += sum(start_pred_label == target_start.squeeze())
    train_end_correct += sum(end_pred_label == target_end.squeeze())

    #loss연산 start에 대한 loss와 end에 대한 loss를 따로 연산해서 더해줌
    loss = lf(start_logit, target_start) + lf(end_logit, target_end)

    loss.backward()

    optimizer.step()

    epoch_loss += loss.item()

  # print(train_start_correct)
  # print(train_end_correct)
  print("train loss", epoch_loss/len(train_dataloader))
  print("train start acc", train_start_correct/len(train_dataset))
  print("train end acc", train_end_correct/len(train_dataset))

  test_loss = 0
  test_start_correct = 0
  test_end_correct = 0

  model.eval()

  with torch.no_grad():
    for batch in test_dataloader:
      input_ids, token_type_ids, attention_mask, target_start, target_end= batch
      input_ids = input_ids.cuda()
      token_type_ids = token_type_ids.cuda()
      attention_mask = attention_mask.cuda()
      target_start = target_start.cuda()
      target_end = target_end.cuda()

      start_logit, end_logit = model(input_ids, token_type_ids, attention_mask)

      start_pred_label = torch.argmax(start_logit, dim=-1)
      end_pred_label = torch.argmax(end_logit, dim=-1)

      test_start_correct += sum(start_pred_label == target_start.reshape(-1))
      test_end_correct += sum(end_pred_label == target_end.reshape(-1))

      #loss연산 start에 대한 loss와 end에 대한 loss를 따로 연산해서 더해줌
      loss = lf(start_logit, target_start) + lf(end_logit, target_end)

      test_loss += loss.item()

  print("test loss", test_loss/len(test_dataloader))
  print("test start acc", test_start_correct/len(test_dataset))
  print("test end acc", test_end_correct/len(test_dataset))




epoch  0


  0%|          | 0/90 [00:00<?, ?it/s]

train loss 9.012169075012206
train start acc tensor(0.0818, device='cuda:0')
train end acc tensor(0.0929, device='cuda:0')
test loss 7.866385225013451
test start acc tensor(0.1605, device='cuda:0')
test end acc tensor(0.0914, device='cuda:0')

epoch  1


  0%|          | 0/90 [00:00<?, ?it/s]

train loss 6.405830483966404
train start acc tensor(0.2082, device='cuda:0')
train end acc tensor(0.2268, device='cuda:0')
test loss 8.657890496430573
test start acc tensor(0.1309, device='cuda:0')
test end acc tensor(0.1309, device='cuda:0')

epoch  2


...

train loss 0.004514655676535837
train start acc tensor(1., device='cuda:0')
train end acc tensor(1., device='cuda:0')
test loss 19.976840504010518
test start acc tensor(0.0975, device='cuda:0')
test end acc tensor(0.0778, device='cuda:0')

epoch  98


  0%|          | 0/90 [00:00<?, ?it/s]

train loss 0.02730087664631229
train start acc tensor(1., device='cuda:0')
train end acc tensor(0.9963, device='cuda:0')
test loss 19.086522655133848
test start acc tensor(0.0951, device='cuda:0')
test end acc tensor(0.0778, device='cuda:0')

epoch  99


  0%|          | 0/90 [00:00<?, ?it/s]

train loss 0.00705389808932624
train start acc tensor(1., device='cuda:0')
train end acc tensor(1., device='cuda:0')
test loss 19.291824605729843
test start acc tensor(0.0938, device='cuda:0')
test end acc tensor(0.0790, device='cuda:0')
